# TPC-H

TPC-H is a decision support benchmark that offers business-oriented ad hoc queries.
More information can be found [here](http://www.tpc.org/tpch)

The queries are originally in SQL format and here they are implemented using the pandas API.

### Notes on running these queries:

By defaults runs use Bodo. Hence, data is distributed in chunks across processes.

The current results are based on running on one **m5.8xlarge** instance (16 cores, 128GiB memory)

Dataset size is 2GB.

There's another dataset available on "s3://bodo-examples-data/tpch/s4/" which is 4GB.

To run the code:
1. Make sure you add your AWS account credentials to access the data. 
2. If you want to run a query in regular pandas:
    1. Comment magic expression (`%%px`) from AWS, import statements, its corresponding functions to load data, and the query cells.
    2. Comment bodo decorator (`@bodo.jit`) from the load_data functions and the query that you'll run. 
    3. Re-run the import cells.
    4. Re-run functions in [loading data](#loading_data) section to have the requried dataset in one process. 


In [5]:
%%px
import json
import os

path_to_conn_creds = "credentials.json"
with open(path_to_conn_creds) as f:
    creds = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = creds["aws"]["aws_access_key_id"]
os.environ["AWS_SECRET_ACCESS_KEY"] = creds["aws"]["aws_secret_access_key"]
os.environ["AWS_DEFAULT_REGION"] = "us-east-2"

In [6]:
%%px
import bodo
import time
import numpy as np
import pandas as pd

<a id="loading_data"></a>
## Loading data

In this section, we load the data required by the queries in pandas DataFrame.

In [7]:
%%px
@bodo.jit(distributed=["rel"], cache=True)
def load_lineitem(data_folder):
    t1 = time.time()
    file = data_folder + "/lineitem.tbl"
    cols_names = ['L_ORDERKEY' , 'L_PARTKEY', 'L_SUPPKEY', 'L_LINENUMBER', 'L_QUANTITY',
            'L_EXTENDEDPRICE', 'L_DISCOUNT', 'L_TAX', 'L_RETURNFLAG', 'L_LINESTATUS', 'L_SHIPDATE',
            'L_COMMITDATE', 'L_RECEIPTDATE', 'L_SHIPINSTRUCT', 'L_SHIPMODE', 'L_COMMENT']
    cols = {'L_ORDERKEY' : np.int64, 'L_PARTKEY' : np.int64, 'L_SUPPKEY' : np.int64, 'L_LINENUMBER' : np.int64, 'L_QUANTITY' : np.float64,
            'L_EXTENDEDPRICE' : np.float64, 'L_DISCOUNT' : np.float64, 'L_TAX' : np.float64, 'L_RETURNFLAG' : str, 'L_LINESTATUS' : str, 'L_SHIPDATE' : str,
            'L_COMMITDATE' : str, 'L_RECEIPTDATE' : str, 'L_SHIPINSTRUCT' : str, 'L_SHIPMODE' : str, 'L_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols,
        parse_dates=[10, 11, 12]
        )
    print("Lineitem Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel

lineitem = load_lineitem("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Lineitem Reading time:  6402.453125488705  (ms)
   L_ORDERKEY  L_PARTKEY  L_SUPPKEY  L_LINENUMBER  L_QUANTITY  \
0           1     310379      15395             1        17.0   
1           1     134619      14620             2        36.0   
2           1     127400       7401             3         8.0   
3           1       4263       9264             4        28.0   
4           1      48054       3061             5        24.0   

   L_EXTENDEDPRICE  L_DISCOUNT  L_TAX L_RETURNFLAG L_LINESTATUS L_SHIPDATE  \
0         23619.12        0.04   0.02            N            O 1996-03-13   
1         59529.96        0.09   0.06            N            O 1996-04-12   
2         11419.20        0.10   0.02            N            O 1996-01-29   
3         32683.28        0.09   0.06            N            O 1996-04-21   
4         24049.20        0.10   0.04            N            O 1996-03-30   

  L_COMMITDATE L_RECEIPTDATE     L_SHIPINSTRUCT L_SHIPMODE  \
0   1996-02-12    1

In [8]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_orders(data_folder):
    t1 = time.time()    
    file = data_folder + "/orders.tbl"
    cols_names = ['O_ORDERKEY', 'O_CUSTKEY', 'O_ORDERSTATUS',
            'O_TOTALPRICE', 'O_ORDERDATE', 'O_ORDERPRIORITY',
            'O_CLERK', 'O_SHIPPRIORITY', 'O_COMMENT']
    cols = {'O_ORDERKEY' : np.int64, 'O_CUSTKEY' : np.int64, 'O_ORDERSTATUS' : str,
            'O_TOTALPRICE' : np.float64, 'O_ORDERDATE' : np.int64, 'O_ORDERPRIORITY' : str,
            'O_CLERK' : str, 'O_SHIPPRIORITY' : np.int64, 'O_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols,
        parse_dates=[4]
        )
    print("Orders Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel
    
orders = load_orders("s3://bodo-examples-data/tpch/s2")

[stdout:0] Orders Reading time:  1366.1140216275953  (ms)
   O_ORDERKEY  O_CUSTKEY O_ORDERSTATUS  O_TOTALPRICE O_ORDERDATE  \
0           1      73801             O     181503.69  1996-01-02   
1           2     156004             O      49967.96  1996-12-01   
2           3     246628             F     227024.64  1993-10-14   
3           4     273553             O      36018.68  1995-10-11   
4           5      88970             F     112288.43  1994-07-30   

  O_ORDERPRIORITY          O_CLERK  O_SHIPPRIORITY  \
0           5-LOW  Clerk#000001902               0   
1        1-URGENT  Clerk#000001759               0   
2           5-LOW  Clerk#000001909               0   
3           5-LOW  Clerk#000000247               0   
4           5-LOW  Clerk#000001850               0   

                                           O_COMMENT  
0                 nstructions sleep furiously among   
1   foxes. pending accounts at the pending, silen...  
2  sly final accounts boost. carefully regu

In [9]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_customer(data_folder):
    t1 = time.time()
    file = data_folder + "/customer.tbl"
    cols_names = ['C_CUSTKEY', 'C_NAME',
            'C_ADDRESS', 'C_NATIONKEY',
            'C_PHONE', 'C_ACCTBAL',
            'C_MKTSEGMENT', 'C_COMMENT']
    cols = {'C_CUSTKEY' : np.int64, 'C_NAME' : str,
            'C_ADDRESS' : str, 'C_NATIONKEY' : np.int64,
            'C_PHONE' : str, 'C_ACCTBAL' : np.float64,
            'C_MKTSEGMENT' : str, 'C_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols
        )
    print("Customer Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel

customer = load_customer("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Customer Reading time:  399.84531761672315  (ms)
   C_CUSTKEY              C_NAME                       C_ADDRESS  C_NATIONKEY  \
0          1  Customer#000000001               IVhzIApeRb ot,c,E           15   
1          2  Customer#000000002  XSTf4,NCwDVaWNe6tEgvwfmRchLXak           13   
2          3  Customer#000000003                    MG9kdTD2WBHm            1   
3          4  Customer#000000004                     XxVSJsLAGtn            4   
4          5  Customer#000000005    KvpyuHCplrB84WgAiGV6sYpZq7Tj            3   

           C_PHONE  C_ACCTBAL C_MKTSEGMENT  \
0  25-989-741-2988     711.56     BUILDING   
1  23-768-687-3665     121.65   AUTOMOBILE   
2  11-719-748-3364    7498.12   AUTOMOBILE   
3  14-128-190-5944    2866.83    MACHINERY   
4  13-750-942-6364     794.47    HOUSEHOLD   

                                           C_COMMENT  
0  to the even, regular platelets. regular, ironi...  
1  l accounts. blithely ironic theodolites integr...  
2   deposit

In [10]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_nation(data_folder):
    t1 = time.time()
    file = data_folder + "/nation.tbl"
    cols_names = ['N_NATIONKEY', 'N_NAME',
            'N_REGIONKEY', 'N_COMMENT']
    cols = {'N_NATIONKEY' : np.int64, 'N_NAME' : str,
            'N_REGIONKEY' : np.int64, 'N_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols
        )
    print("Nation Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel

nation = load_nation("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:2]    N_NATIONKEY N_NAME  N_REGIONKEY  \
4            4  EGYPT            4   

                                           N_COMMENT  
4  y above the carefully unusual theodolites. fin...  


[stdout:0] Nation Reading time:  116.6650568050045  (ms)
   N_NATIONKEY     N_NAME  N_REGIONKEY  \
0            0    ALGERIA            0   
1            1  ARGENTINA            1   

                                           N_COMMENT  
0   haggle. carefully final deposits detect slyly...  
1  al foxes promise slyly according to the regula...  


[stdout:1]    N_NATIONKEY  N_NAME  N_REGIONKEY  \
2            2  BRAZIL            1   
3            3  CANADA            1   

                                           N_COMMENT  
2  y alongside of the pending deposits. carefully...  
3  eas hang ironic, silent packages. slyly regula...  


In [11]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_supplier(data_folder):
    t1 = time.time()    
    file = data_folder + "/supplier.tbl"
    cols_names = ['S_SUPPKEY', 'S_NAME', 'S_ADDRESS',
            'S_NATIONKEY', 'S_PHONE', 'S_ACCTBAL',
            'S_COMMENT']
    cols = {'S_SUPPKEY' : np.int64, 'S_NAME' : str, 'S_ADDRESS' : str,
            'S_NATIONKEY' : np.int64, 'S_PHONE' : str, 'S_ACCTBAL' : np.float64,
            'S_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols
        )
    print("Supplier Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())        
    return rel

supplier = load_supplier("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Supplier Reading time:  213.53314048349148  (ms)
   S_SUPPKEY              S_NAME                            S_ADDRESS  \
0          1  Supplier#000000001   N kD4on9OM Ipw3,gf0JBoQDd7tgrzrddZ   
1          2  Supplier#000000002                89eJ5ksX3ImxJQBvxObC,   
2          3  Supplier#000000003    q1,G3Pj6OjIuUYfUoH18BFTKP5aU9bEV3   
3          4  Supplier#000000004            Bk7ah4CK8SYQTepEmvMkkgMwg   
4          5  Supplier#000000005                    Gcdm2rJRzl5qlTVzc   

   S_NATIONKEY          S_PHONE  S_ACCTBAL  \
0           17  27-918-335-1736    5755.94   
1            5  15-679-861-2259    4032.68   
2            1  11-383-516-1199    4192.40   
3           15  25-843-787-7479    4641.08   
4           11  21-151-690-3663    -283.84   

                                           S_COMMENT  
0                       each slyly above the careful  
1             slyly bold instructions. idle dependen  
2  blithely silent requests after the express dep...  
3   

In [12]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_partsupp(data_folder):
    t1 = time.time()
    file = data_folder + "/partsupp.tbl"
    cols_names = ['PS_PARTKEY', 'PS_SUPPKEY', 'PS_AVAILQTY',
            'PS_SUPPLYCOST', 'PS_COMMENT']
    cols = {'PS_PARTKEY' : np.int64, 'PS_SUPPKEY' : np.int64, 'PS_AVAILQTY' : np.int64,
            'PS_SUPPLYCOST' : np.float64, 'PS_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols
        )
    print("Partsupp Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel

partsupp = load_partsupp("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Partsupp Reading time:  1039.3468976068334  (ms)
   PS_PARTKEY  PS_SUPPKEY  PS_AVAILQTY  PS_SUPPLYCOST  \
0           1           2         3325         771.64   
1           1        5002         8076         993.49   
2           1       10002         3956         337.09   
3           1       15002         4069         357.84   
4           2           3         8895         378.49   

                                          PS_COMMENT  
0  , even theodolites. regular, final theodolites...  
1  ven ideas. quickly even packages print. pendin...  
2  after the fluffily ironic deposits? blithely s...  
3  al, regular dependencies serve carefully after...  
4  nic accounts. final accounts sleep furiously a...  


In [13]:
%%px

@bodo.jit(distributed=["rel"], cache=True)
def load_part(data_folder):
    t1 = time.time()
    file = data_folder + "/part.tbl"
    cols_names = ['P_PARTKEY', 'P_NAME', 'P_MFGR', 'P_BRAND',
            'P_TYPE', 'P_SIZE', 'P_CONTAINER',
            'P_RETAILPRICE', 'P_COMMENT']
    cols = {'P_PARTKEY' : np.int64, 'P_NAME' : str, 'P_MFGR' : str, 'P_BRAND' : str,
            'P_TYPE' : str, 'P_SIZE' : np.int64, 'P_CONTAINER' : str,
            'P_RETAILPRICE' : np.float64, 'P_COMMENT' : str}
    rel = pd.read_csv(file, sep='|', header=None,
        names=cols_names,
        dtype=cols
        )
    print("Part Reading time: ", ((time.time() - t1) * 1000), " (ms)")
    print(rel.head())
    return rel

part = load_part("s3://bodo-examples-data/tpch/s2")

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Part Reading time:  457.4376229434165  (ms)
   P_PARTKEY                                    P_NAME          P_MFGR  \
0          1  goldenrod lavender spring chocolate lace  Manufacturer#1   
1          2          blush thistle blue yellow saddle  Manufacturer#1   
2          3       spring green yellow purple cornsilk  Manufacturer#4   
3          4     cornflower chocolate smoke green pink  Manufacturer#3   
4          5             forest brown coral puff cream  Manufacturer#3   

    P_BRAND                   P_TYPE  P_SIZE P_CONTAINER  P_RETAILPRICE  \
0  Brand#13   PROMO BURNISHED COPPER       7   JUMBO PKG          901.0   
1  Brand#13      LARGE BRUSHED BRASS       1     LG CASE          902.0   
2  Brand#42  STANDARD POLISHED BRASS      21   WRAP CASE          903.0   
3  Brand#34       SMALL PLATED BRASS      14    MED DRUM          904.0   
4  Brand#32    STANDARD POLISHED TIN      15      SM PKG          905.0   

             P_COMMENT  
0      ly. slyly ironi  

## Query Definitions

This section includes some of the queries using Python (Pandas)

### Q1: Pricing Summary Report Query
This query reports the amount of businesses that were billed, shipped, and returned.

Make sure you have run **`load_lineitem`** from [loading data section](#loading_data) before running this query.

In [14]:
%%px
@bodo.jit(cache=True)
def q1(lineitem):
    t1 = time.time()
    sel = lineitem.L_SHIPDATE <= "1998-09-02"
    flineitem = lineitem[sel]
    flineitem["DISC_PRICE"] = flineitem.L_EXTENDEDPRICE * (1 - flineitem.L_DISCOUNT)
    flineitem["CHARGE"] = (
        flineitem.L_EXTENDEDPRICE * (1 - flineitem.L_DISCOUNT) * (1 + flineitem.L_TAX)
    )
    gb = flineitem.groupby(["L_RETURNFLAG", "L_LINESTATUS"], as_index=False)
    total = gb.agg({"L_QUANTITY": ["sum", "mean"], "L_EXTENDEDPRICE": ["sum", "mean"],
                   "DISC_PRICE": "sum", "CHARGE": "sum",
                   "L_DISCOUNT": "mean", "L_ORDERKEY": "count"})
    total = total.sort_values(["L_RETURNFLAG", "L_LINESTATUS"])
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q1_result = q1(lineitem)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:8] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:15] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:10] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:3] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


...

[stdout:7] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:11] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:2] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:1] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


[stdout:13] Empty DataFrame
Columns: [(L_RETURNFLAG, ), (L_LINESTATUS, ), (L_QUANTITY, sum), (L_QUANTITY, mean), (L_EXTENDEDPRICE, sum), (L_EXTENDEDPRICE, mean), (DISC_PRICE, sum), (CHARGE, sum), (L_DISCOUNT, mean), (L_ORDERKEY, count)]
Index: []


### Q3: Shipping Priority Query
This query retrieves the 10 unshipped orders with the highest value.

Make sure you have run **`load_lineitem`, `load_orders`, and `load_customer`** from [loading data section](#loading_data) before running this query.

In [15]:
%%px

@bodo.jit(cache=True)
def q3(lineitem, orders, customer):
    date = "1995-03-04"    
    t1 = time.time()
    lsel = lineitem.L_SHIPDATE > date
    osel = orders.O_ORDERDATE < date
    csel = customer.C_MKTSEGMENT == "HOUSEHOLD"
    flineitem = lineitem[lsel]
    forders = orders[osel]
    fcustomer = customer[csel]
    jn1 = fcustomer.merge(forders, left_on="C_CUSTKEY", right_on="O_CUSTKEY")
    jn2 = jn1.merge(flineitem, left_on="O_ORDERKEY", right_on="L_ORDERKEY")

    jn2["TMP"] = jn2.L_EXTENDEDPRICE * (1 - jn2.L_DISCOUNT)

    total = (
        jn2.groupby(
            ["L_ORDERKEY", "O_ORDERDATE", "O_SHIPPRIORITY"], as_index=False
        )["TMP"]
        .sum()
        .sort_values(["TMP"], ascending=False)
    )
    res = total[["L_ORDERKEY", "TMP", "O_ORDERDATE", "O_SHIPPRIORITY"]]

    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(res.head(10))
    return res.head(10)

q3_result = q3(lineitem, orders, customer)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Execution time:  123.560858952942  (ms)
       L_ORDERKEY          TMP O_ORDERDATE  O_SHIPPRIORITY
22385    11495873  417031.8957  1995-01-17               0
1890      4163074  416152.8027  1995-02-13               0
14641     6487431  412710.0508  1995-02-06               0
10297     5536290  407857.9678  1995-01-31               0
19514    10666915  397407.4185  1995-02-14               0
17217     4225253  391852.7649  1995-02-28               0
6285      6232772  391241.2513  1995-01-17               0
21609     4724865  387185.9724  1995-02-03               0
3261      3900355  386089.5768  1995-02-13               0
21333     2377474  383057.9763  1995-02-26               0


[stdout:8] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:6] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:4] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


...

[stdout:15] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:5] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:10] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:12] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


[stdout:7] Empty DataFrame
Columns: [L_ORDERKEY, TMP, O_ORDERDATE, O_SHIPPRIORITY]
Index: []


### Q4: Order Priority Checking Query
This query determines how well the order priority system is working and gives an assessment of customer satisfaction.

Make sure you have run **`load_lineitem` and `load_orders`** from [loading data section](#loading_data) before running this query.

In [16]:
%%px
@bodo.jit(cache=True)
def q4(lineitem, orders):
    date1 = "1993-11-01"
    date2 = "1993-08-01"
    t1 = time.time()
    lsel = lineitem.L_COMMITDATE < lineitem.L_RECEIPTDATE
    osel = (orders.O_ORDERDATE < date1) & (orders.O_ORDERDATE >= date2)
    flineitem = lineitem[lsel]
    forders = orders[osel]
    jn = forders[forders["O_ORDERKEY"].isin(flineitem["L_ORDERKEY"])]
    total = (
        jn.groupby("O_ORDERPRIORITY", as_index=False)["O_ORDERKEY"]
        .count()
        .sort_values(["O_ORDERPRIORITY"])
    )
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q4_result = q4(lineitem, orders)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:4] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:6] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:11] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:12] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


...

[stdout:9] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:7] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:1] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:15] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


[stdout:5] Empty DataFrame
Columns: [O_ORDERPRIORITY, O_ORDERKEY]
Index: []


### Q6: Forecasting Revenue Change Query
This query quantifies the amount of revenue increase that would have resulted from eliminating certain company-wide discounts in a given percentage range in a given year.

Make sure you have run **`load_lineitem`** from [loading data section](#loading_data) before running this query.

In [17]:
%%px
@bodo.jit(cache=True)
def q6(lineitem):
    date1 = "1996-01-01"
    date2 = "1997-01-01"
    t1 = time.time()
    sel = (
        (lineitem.L_SHIPDATE >= date1)
        & (lineitem.L_SHIPDATE < date2)
        & (lineitem.L_DISCOUNT >= 0.08)
        & (lineitem.L_DISCOUNT <= 0.1)
        & (lineitem.L_QUANTITY < 24)
    )
    flineitem = lineitem[sel]
    total = (flineitem.L_EXTENDEDPRICE * flineitem.L_DISCOUNT).sum()
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total)
    return total

q6_result = q6(lineitem)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Execution time:  23.68376558149521  (ms)
369930968.9013001


### Q9: Product Type Profit Measure Query
This query determines how much profit is made on a given line of parts, broken out by supplier nation and year.

Make sure you have run **`load_lineitem`, `load_orders`, `load_part`, `load_nation`, `load_partsupp`, and `load_supplier`** from [loading data section](#loading_data) before running this query.

In [18]:
%%px
@bodo.jit(cache=True)
def q9(lineitem, orders, part, nation, partsupp, supplier):
    t1 = time.time()
    psel = part.P_NAME.str.contains("ghost")
    fpart = part[psel]
    jn1 = lineitem.merge(fpart, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jn2 = jn1.merge(supplier, left_on="L_SUPPKEY", right_on="S_SUPPKEY")
    jn3 = jn2.merge(nation, left_on="S_NATIONKEY", right_on="N_NATIONKEY")
    jn4 = partsupp.merge(
        jn3, left_on=["PS_PARTKEY", "PS_SUPPKEY"], right_on=["L_PARTKEY", "L_SUPPKEY"]
    )
    jn5 = jn4.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn5["TMP"] = jn5.L_EXTENDEDPRICE * (1 - jn5.L_DISCOUNT) - (
        (1 * jn5.PS_SUPPLYCOST) * jn5.L_QUANTITY
    )
    jn5["O_YEAR"] = jn5.O_ORDERDATE.dt.year
    gb = jn5.groupby(["N_NAME", "O_YEAR"], as_index=False)["TMP"].sum()
    total = gb.sort_values(["N_NAME", "O_YEAR"], ascending=[True, False])
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q9_result = q9(lineitem, orders, part, nation, partsupp, supplier)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:9] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:4] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:6] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:11] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


...

[stdout:8] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:7] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:5] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:12] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


[stdout:2] Empty DataFrame
Columns: [N_NAME, O_YEAR, TMP]
Index: []


### Q10: Returned Item Reporting Query
This query identifies customers who might be having problems with the parts that are shipped to them.

Make sure you have run **`load_lineitem`, `load_orders`, `load_customer`, and `load_nation`** from [loading data section](#loading_data) before running this query.

In [19]:
%%px
@bodo.jit(cache=True)
def q10(lineitem, orders, customer, nation):
    date1 = "1994-11-01"
    date2 = "1995-02-01"
    t1 = time.time()
    osel = (orders.O_ORDERDATE >= date1) & (orders.O_ORDERDATE < date2)
    lsel = lineitem.L_RETURNFLAG == "R"
    forders = orders[osel]
    flineitem = lineitem[lsel]
    jn1 = flineitem.merge(forders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn2 = jn1.merge(customer, left_on="O_CUSTKEY", right_on="C_CUSTKEY")
    jn3 = jn2.merge(nation, left_on="C_NATIONKEY", right_on="N_NATIONKEY")
    jn3["TMP"] = jn3.L_EXTENDEDPRICE * (1.0 - jn3.L_DISCOUNT)
    gb = jn3.groupby(
        [
            "C_CUSTKEY",
            "C_NAME",
            "C_ACCTBAL",
            "C_PHONE",
            "N_NAME",
            "C_ADDRESS",
            "C_COMMENT",
        ],
        as_index=False,
    )["TMP"].sum()
    total = gb.sort_values("TMP", ascending=False)
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q10_result = q10(lineitem, orders, customer, nation)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:9] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:15] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:7] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:14] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


...

[stdout:2] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:10] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:0] Execution time:  126.95406935631581  (ms)
       C_CUSTKEY              C_NAME  C_ACCTBAL          C_PHONE  \
1588      202000  Customer#000202000    3155.77  30-860-645-7227   
55889     117250  Customer#000117250    8260.39  17-524-241-3788   
59921      67172  Customer#000067172    1747.27  21-578-917-6336   
28889      87863  Customer#000087863    3105.23  21-393-302-3317   
57000     284104  Customer#000284104    2668.71  32-167-107-4631   
25252     168475  Customer#000168475    3873.61  21-537-772-3811   
10722     217825  Customer#000217825    1753.84  23-765-943-1680   
7556      288742  Customer#000288742    6316.82  12-549-320-8899   
29454      50281  Customer#000050281    3306.39  25-210-337-8539   
45383     159304  Customer#000159304    -342.39  18-681-863-8034   

             N_NAME                             C_ADDRESS  \
1588   SAUDI ARABIA                        RITU1eYat8iNeD   
55889       GERMANY                2N bS9peD0b5Dr3tf Vbxq   
59921          

[stdout:5] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


[stdout:13] Empty DataFrame
Columns: [C_CUSTKEY, C_NAME, C_ACCTBAL, C_PHONE, N_NAME, C_ADDRESS, C_COMMENT, TMP]
Index: []


### Q12: Shipping Modes and Order Priority Query
This query determines whether selecting less expensive modes of shipping is negatively affecting the critical-priority orders by causing more parts to be received by customers after the committed date.

Make sure you have run **`load_lineitem` and `load_orders`** from [loading data section](#loading_data) before running this query.

In [20]:
%%px
@bodo.jit(cache=True)
def q12(lineitem, orders):
    date1 = "1994-01-01"
    date2 = "1995-01-01"
    t1 = time.time()
    sel = (
        (lineitem.L_RECEIPTDATE < date2)
        & (lineitem.L_COMMITDATE < date2)
        & (lineitem.L_SHIPDATE < date2)
        & (lineitem.L_SHIPDATE < lineitem.L_COMMITDATE)
        & (lineitem.L_COMMITDATE < lineitem.L_RECEIPTDATE)
        & (lineitem.L_RECEIPTDATE >= date1)
        & ((lineitem.L_SHIPMODE == "MAIL") | (lineitem.L_SHIPMODE == "SHIP"))
    )
    flineitem = lineitem[sel]
    jn = flineitem.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")

    def g1(x):
        return ((x == "1-URGENT") | (x == "2-HIGH")).sum()

    def g2(x):
        return ((x != "1-URGENT") & (x != "2-HIGH")).sum()

    total = jn.groupby("L_SHIPMODE", as_index=False)["O_ORDERPRIORITY"].agg((g1, g2))
    total = total.sort_values("L_SHIPMODE")
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q12_result = q12(lineitem, orders)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Execution time:  113.19822869893414  (ms)
  L_SHIPMODE     g1     g2
0       MAIL  12354  18548
1       SHIP  12430  18644


[stdout:9] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:13] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:8] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


...

[stdout:7] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:14] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:11] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:12] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


[stdout:5] Empty DataFrame
Columns: [L_SHIPMODE, g1, g2]
Index: []


### Q14: Promotion Effect Query
This query monitors the market response to a promotion such as TV advertisements or a special campaign.

Make sure you have run **`load_lineitem`** and **`load_part`** from [loading data section](#loading_data) before running this query.

In [21]:
%%px
@bodo.jit(cache=True)
def q14(lineitem, part):
    startDate = "1994-03-01"
    endDate = "1994-04-01"
    p_type_like = "PROMO"
    t1 = time.time()
    sel = (lineitem.L_SHIPDATE >= startDate) & (lineitem.L_SHIPDATE < endDate)
    flineitem = lineitem[sel]
    jn = flineitem.merge(part, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jn["TMP"] = jn.L_EXTENDEDPRICE * (1.0 - jn.L_DISCOUNT)
    total = jn[jn.P_TYPE.str.startswith(p_type_like)].TMP.sum() * 100 / jn.TMP.sum()
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total)
    return total

q14_result = q14(lineitem, part)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Execution time:  37.17108614659992  (ms)
16.545810644547483


### Q18: Large Volume Customer Query
This query ranks customers based on their having placed a large quantity order. Large quantity orders are defined as those orders whose total quantity is above a certain level.

Make sure you have run **`load_lineitem`, `load_orders`, and `load_customer`** from [loading data section](#loading_data) before running this query.

In [22]:
%%px
@bodo.jit(cache=True)
def q18(lineitem, orders, customer):
    t1 = time.time()
    gb1 = lineitem.groupby("L_ORDERKEY", as_index=False)["L_QUANTITY"].sum()
    fgb1 = gb1[gb1.L_QUANTITY > 300]
    jn1 = fgb1.merge(orders, left_on="L_ORDERKEY", right_on="O_ORDERKEY")
    jn2 = jn1.merge(customer, left_on="O_CUSTKEY", right_on="C_CUSTKEY")
    gb2 = jn2.groupby(
        ["C_NAME", "C_CUSTKEY", "O_ORDERKEY", "O_ORDERDATE", "O_TOTALPRICE"],
        as_index=False,
    )["L_QUANTITY"].sum()
    total = gb2.sort_values(["O_TOTALPRICE", "O_ORDERDATE"], ascending=[False, True])
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q18_result = q18(lineitem, orders, customer)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:1]                 C_NAME  C_CUSTKEY  O_ORDERKEY O_ORDERDATE  O_TOTALPRICE  \
95  Customer#000176644     176644    10889601  1992-05-22     513824.71   
57  Customer#000258475     258475     7125602  1994-07-08     509018.85   

    L_QUANTITY  
95       306.0  
57       301.0  


[stdout:6] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


[stdout:0] Execution time:  86.2612184300815  (ms)
                 C_NAME  C_CUSTKEY  O_ORDERKEY O_ORDERDATE  O_TOTALPRICE  \
113  Customer#000256240     256240     4722021  1994-04-07     543948.47   
47   Customer#000192203     192203     5984582  1992-03-16     539085.20   
4    Customer#000273004     273004    11785570  1996-07-18     535097.55   
67   Customer#000198385     198385     8574884  1992-07-04     530902.09   
88   Customer#000186325     186325     9436480  1992-05-22     523925.49   
99   Customer#000048682      48682     1474818  1992-11-15     522718.60   
61   Customer#000027880      27880     2232932  1997-04-13     519887.44   
52   Customer#000082297      82297     8231942  1993-02-06     516398.50   

     L_QUANTITY  
113       323.0  
47        312.0  
4         303.0  
67        308.0  
88        311.0  
99        302.0  
61        304.0  
52        302.0  


[stdout:8] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


...

[stdout:14] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


[stdout:10] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


[stdout:2] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


[stdout:12] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


[stdout:5] Empty DataFrame
Columns: [C_NAME, C_CUSTKEY, O_ORDERKEY, O_ORDERDATE, O_TOTALPRICE, L_QUANTITY]
Index: []


### Q19: Discounted Revenue Query
This query reports the gross discounted revenue attributed to the sale of selected parts handled in a particular manner.

Make sure you have run **`load_lineitem`** and **`load_part`** from [loading data section](#loading_data) before running this query.

In [23]:
%%px
@bodo.jit(cache=True)
def q19(lineitem, part):
    Brand31 = "Brand#31"
    Brand43 = "Brand#43"
    SMBOX = "SM BOX"
    SMCASE = "SM CASE"
    SMPACK = "SM PACK"
    SMPKG = "SM PKG"
    MEDBAG = "MED BAG"
    MEDBOX = "MED BOX"
    MEDPACK = "MED PACK"
    MEDPKG = "MED PKG"
    LGBOX = "LG BOX"
    LGCASE = "LG CASE"
    LGPACK = "LG PACK"
    LGPKG = "LG PKG"
    DELIVERINPERSON = "DELIVER IN PERSON"
    AIR = "AIR"
    AIRREG = "AIRREG"
    t1 = time.time()
    lsel = (
        (
            ((lineitem.L_QUANTITY <= 36) & (lineitem.L_QUANTITY >= 26))
            | ((lineitem.L_QUANTITY <= 25) & (lineitem.L_QUANTITY >= 15))
            | ((lineitem.L_QUANTITY <= 14) & (lineitem.L_QUANTITY >= 4))
        )
        & (lineitem.L_SHIPINSTRUCT == DELIVERINPERSON)
        & ((lineitem.L_SHIPMODE == AIR) | (lineitem.L_SHIPMODE == AIRREG))
    )
    psel = (part.P_SIZE >= 1) & (
        (
            (part.P_SIZE <= 5)
            & (part.P_BRAND == Brand31)
            & (
                (part.P_CONTAINER == SMBOX)
                | (part.P_CONTAINER == SMCASE)
                | (part.P_CONTAINER == SMPACK)
                | (part.P_CONTAINER == SMPKG)
            )
        )
        | (
            (part.P_SIZE <= 10)
            & (part.P_BRAND == Brand43)
            & (
                (part.P_CONTAINER == MEDBAG)
                | (part.P_CONTAINER == MEDBOX)
                | (part.P_CONTAINER == MEDPACK)
                | (part.P_CONTAINER == MEDPKG)
            )
        )
        | (
            (part.P_SIZE <= 15)
            & (part.P_BRAND == Brand43)
            & (
                (part.P_CONTAINER == LGBOX)
                | (part.P_CONTAINER == LGCASE)
                | (part.P_CONTAINER == LGPACK)
                | (part.P_CONTAINER == LGPKG)
            )
        )
    )
    flineitem = lineitem[lsel]
    fpart = part[psel]
    jn = flineitem.merge(fpart, left_on="L_PARTKEY", right_on="P_PARTKEY")
    jnsel = (
        (jn.P_BRAND == Brand31)
        & (
            (jn.P_CONTAINER == SMBOX)
            | (jn.P_CONTAINER == SMCASE)
            | (jn.P_CONTAINER == SMPACK)
            | (jn.P_CONTAINER == SMPKG)
        )
        & (jn.L_QUANTITY >= 4)
        & (jn.L_QUANTITY <= 14)
        & (jn.P_SIZE <= 5)
        | (jn.P_BRAND == Brand43)
        & (
            (jn.P_CONTAINER == MEDBAG)
            | (jn.P_CONTAINER == MEDBOX)
            | (jn.P_CONTAINER == MEDPACK)
            | (jn.P_CONTAINER == MEDPKG)
        )
        & (jn.L_QUANTITY >= 15)
        & (jn.L_QUANTITY <= 25)
        & (jn.P_SIZE <= 10)
        | (jn.P_BRAND == Brand43)
        & (
            (jn.P_CONTAINER == LGBOX)
            | (jn.P_CONTAINER == LGCASE)
            | (jn.P_CONTAINER == LGPACK)
            | (jn.P_CONTAINER == LGPKG)
        )
        & (jn.L_QUANTITY >= 26)
        & (jn.L_QUANTITY <= 36)
        & (jn.P_SIZE <= 15)
    )
    jn = jn[jnsel]
    total = (jn.L_EXTENDEDPRICE * (1.0 - jn.L_DISCOUNT)).sum()
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total)
    return total

q19_result = q19(lineitem, part)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:0] Execution time:  94.34347352487293  (ms)
7178285.3134


### Q20: Potential Part Promotion Query
This query identifies suppliers in a particular nation having selected parts that may be candidates for a promotional offer.

Make sure you have run **`load_lineitem`, `load_part`, `load_nation`, `load_partsupp`, and `load_supplier`** from [loading data section](#loading_data) before running this query.

In [24]:
%%px
@bodo.jit(cache=True)
def q20(lineitem, part, nation, partsupp, supplier):
    date1 = "1996-01-01"
    date2 = "1997-01-01"
    t1 = time.time()
    psel = part.P_NAME.str.startswith("azure")
    nsel = nation.N_NAME == "JORDAN"
    lsel = (lineitem.L_SHIPDATE >= date1) & (lineitem.L_SHIPDATE < date2)
    fpart = part[psel]
    fnation = nation[nsel]
    flineitem = lineitem[lsel]
    jn1 = fpart.merge(partsupp, left_on="P_PARTKEY", right_on="PS_PARTKEY")
    jn2 = jn1.merge(
        flineitem,
        left_on=["PS_PARTKEY", "PS_SUPPKEY"],
        right_on=["L_PARTKEY", "L_SUPPKEY"],
    )
    gb = jn2.groupby(["PS_PARTKEY", "PS_SUPPKEY", "PS_AVAILQTY"], as_index=False)[
        "L_QUANTITY"
    ].sum()
    gbsel = gb.PS_AVAILQTY > (0.5 * gb.L_QUANTITY)
    fgb = gb[gbsel]
    jn3 = fgb.merge(supplier, left_on="PS_SUPPKEY", right_on="S_SUPPKEY")
    jn4 = fnation.merge(jn3, left_on="N_NATIONKEY", right_on="S_NATIONKEY")
    jn4 = jn4[["S_NAME", "S_ADDRESS"]]
    total = jn4.sort_values("S_NAME").drop_duplicates()
    print("Execution time: ", ((time.time() - t1) * 1000), " (ms)")
    print(total.head(10))
    return total.head(10)

q20_result = q20(lineitem, part, nation, partsupp, supplier)

%px:   0%|          | 0/16 [00:00<?, ?tasks/s]

[stdout:13] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:15] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:5] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:1] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


...

[stdout:3] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:14] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:11] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:10] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []


[stdout:12] Empty DataFrame
Columns: [S_NAME, S_ADDRESS]
Index: []
